In [10]:
import sys
import os
from pathlib import Path
import re
import numpy as np

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

COMMON = Path('../data/raw/COMMON_images_masks/')
GROUP = Path('../data/raw/GROUP_images/')
DATA = (COMMON.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk
from utils import *
from linear_registration import register_linear

In [19]:
common_names = ['common_40', 'common_41', 'common_42']
group_paths = [file for file in GROUP.iterdir()]
group_pattern = re.compile(r'^(g\d{1,2}_\d{2})_image\.nii\.gz$')

group_names = []

for file in GROUP.iterdir():
    match = group_pattern.match(file.name)
    if match:
        group_names.append(match.group(1))

print(f"{common_names=}")
print(f"{group_names=}")


common_names=['common_40', 'common_41', 'common_42']
group_names=['g3_60', 'g1_54', 'g5_66', 'g8_74', 'g4_64', 'g11_85', 'g11_83', 'g3_59', 'g10_80', 'g1_55', 'g6_68', 'g6_69', 'g12_86', 'g2_58', 'g10_82', 'g8_75', 'g4_63', 'g4_62', 'g9_78', 'g5_65', 'g7_73', 'g2_57', 'g9_79', 'g9_77', 'g7_72', 'g1_56', 'g10_81', 'g7_71']


In [32]:
group_train_vects = []
for path in group_paths:
    im = sitk.ReadImage(path)
    group_train_vects.append(np.zeros(im.GetSize()[2]).astype(int))
    del(im)

In [53]:
len(group_names)

28

In [63]:
im_idx = 8
im = sitk.ReadImage(group_paths[im_idx])
im_arr = sitk.GetArrayFromImage(im)
show_interactive(im_arr,'axial')
print(f'im: {group_names[im_idx]},   nr. axial slices: {group_train_vects[im_idx].size}')

interactive(children=(IntSlider(value=130, description='slc', max=260), Output()), _dom_classes=('widget-inter…

im: g10_80,   nr. axial slices: 261


In [64]:
initial = 52
final = 81
for nr in range (initial, final+1):
    group_train_vects[im_idx][nr] = 1

In [65]:
group_train_vects[im_idx]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [71]:
import pickle

with open (DATA/'training/group_train_vects.pkl', "wb") as f:
    pickle.dump(group_train_vects, f)